# Avaliação de sistema RAG

Este notebook apresenta uma avaliação de desempenho de um sistema de Retrieval Augmented Generation (RAG) com modelos de embeddings diferentes.
Serão testados quatro configurações:

1. Configuração sem RAG (apenas LLM)
2. Configuração com RAG usando modelo de embeddings Gecko (text-embedding-004), da Google
3. Configuração com RAG usando modelo de embeddings all-mpnet-base-v2, da Sentence Transformers Hugging Face
4. Configuração com RAG usando modelo de embeddings all-MiniLM-L6-v2, da Sentence Transformers Hugging Face

## Instalando bibliotecas
Utilizar esta célula se estiver executando no Google Colab

In [ ]:
#!pip install langchain_community langchain_chroma langchain-google-genai chromadb datasets seaborn langchain_huggingface sentence_transformers matplotlib-venn

## Importando bibliotecas

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from rag_pipeline import DocumentProcessingClient, QuestionAnsweringClient
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Instanciando LLM e modelo de embeddings

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

In [ ]:
# Cria uma instância do modelo de inferência e do modelo de embeddings usando a API do LangChain
embedding_model_google = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",google_api_key=config['GOOGLE']['GOOGLE_API_KEY'])
embedding_model_st = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
embedding_model_st_mini = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

llm_model_llama = HuggingFaceEndpoint(model='meta-llama/Llama-3.2-1B',task='text-generation',huggingfacehub_api_token=config['HUGGINGFACE']['HF_TOKEN'])

In [ ]:
# Verificando o tamanho dos embeddings
sample_embedding_1 = embedding_model_google.embed_query('What is Uruguay country?')
sample_embedding_2 = embedding_model_st.embed_query('What is Uruguay?')
sample_embedding_3 = embedding_model_st_mini.embed_query('What is Uruguay?')
print(f'Cada embedding tem as seguintes dimensões:\nGoogle text-embedding-004: {len(sample_embedding_1)} dimensões\nSentence Transformers all-mpnet-base-v2: {len(sample_embedding_2)} dimensões\nSentence Transformers all-MiniLM-L6-v2: {len(sample_embedding_3)} dimensões')

## Importando os dados

Para este trabalho, usaremos apenas o conjunto de desenvolvimento do [SQuAD2.0](https://rajpurkar.github.io/SQuAD-explorer/) que possui em torno de 11800 pares de perguntas e respostas sobre 1200 documentos.

Este dataset será usado para avaliar o RAG que será desenvolvido.

In [ ]:
import json
with open('squad-set/dev-v2.0.json', 'r') as f:
    squad_data = json.load(f)

In [ ]:
# Ajustando o dataframe do SQuAD
answers_list = []
i=0
doc_num=0
for data in squad_data['data']:
  title = data['title']
  for paragraph in data['paragraphs']:
    context = paragraph['context']
    for qa in paragraph['qas']:
      question = qa['question']
      id = qa['id']
      is_impossible = qa['is_impossible']
      if is_impossible:
        answers = qa['plausible_answers']
      else:
        answers = qa['answers']
      answers_list.append({'title': title, 'context': context, 'doc_num': doc_num, 'question': question, 'question_id': id, 'is_impossible': is_impossible, 'answers': answers})
      print(f'{i} lines processed', end='\r')
      i+=1
    doc_num+=1

In [ ]:
squad_dataset = pd.DataFrame(answers_list)

In [ ]:
squad_dataset.head(15)

In [ ]:
squad_dataset.shape

In [ ]:
plt.figure(figsize=(10,6))
p = sns.barplot(x=squad_dataset['title'].value_counts().index, y=squad_dataset['title'].value_counts(), color='green')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=90)
plt.xlabel('Área de conhecimento')
plt.ylabel('Número de perguntas')
plt.title('Número de perguntas por área de conhecimento')
for c in p.containers:
  labels = [int(v.get_height()) for v in c]
  p.bar_label(c, labels=labels, label_type='edge')
plt.show()

O dataset possui 11873 linhas, com cada linha correspondendo a uma pergunta.
Existe mais de uma pergunta para cada contexto.
Cada documento de contexto possui um número `doc_num`.

Algumas perguntas não podem ser respondidas com base no contexto. Elas estão identificadas como `is_impossible == True`.

Cada pergunta possui uma série de respostas aceitáveis.

In [ ]:
# Verificando o número de perguntas cuja resposta não se encontra no texto.
squad_dataset.is_impossible.value_counts()

## Criando vectorstore

In [ ]:
# Carregando os dados do corpus de texto para Langchain Documents
from langchain_community.document_loaders import DataFrameLoader
context_df = squad_dataset[['title','context', 'doc_num']].drop_duplicates(subset=['context']).reset_index()
docs = DataFrameLoader(data_frame=context_df,page_content_column="context").load()
print(f'Número de documentos: {len(docs)}')

In [ ]:
from rag_pipeline import DocumentProcessingClient

doc_processor_google = DocumentProcessingClient(embedding_model=embedding_model_google)
doc_processor_st = DocumentProcessingClient(embedding_model=embedding_model_st)
doc_processor_st_mini = DocumentProcessingClient(embedding_model=embedding_model_st_mini)

In [ ]:
# Transforma os LangChain Documents em embeddings e salva localmente
# O chunk size será definido para 1000
vectorstore_google = doc_processor_google.create_chroma_vectorstore_from_docs(docs=docs, chunk_size=1000, persist_directory="vectorstores/database_google")
vectorstore_st = doc_processor_st.create_chroma_vectorstore_from_docs(docs=docs, chunk_size=1000, persist_directory="vectorstores/database_st")
vectorstore_st_mini = doc_processor_st_mini.create_chroma_vectorstore_from_docs(docs=docs, chunk_size=1000, persist_directory="vectorstores/database_st_mini")

In [ ]:
# Número de documentos após o split
collection = vectorstore_google.get()['metadatas']
print(f'Número de documentos após o split: {len(collection)}')

# Número de documentos por área de conhecimento
dicionario = {}
for doc in collection:
  if doc['title'] not in dicionario:
    dicionario[doc['title']] = 1
  else:
    dicionario[doc['title']] += 1

# Cria um dataframe com as informações para plotar um gráfico
lista = [{'title':title, 'count':count} for title,count in dicionario.items()]
docs_df = pd.DataFrame(lista).sort_values(by='count', ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
p = sns.barplot(data=docs_df, x='title', y='count')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=90)
plt.xlabel('Área de conhecimento')
plt.ylabel('Número de documentos')
plt.title('Número de documentos por área de conhecimento')
for c in p.containers:
  labels = [int(v.get_height()) for v in c]
  p.bar_label(c, labels=labels, label_type='edge')
plt.show()

## Respondendo a perguntas do SQuAD

Serão selecionadas 100 perguntas do dataset para serem respondidas utilizando cada uma das 4 configurações.

Depois de geradas as respostas, elas serão compiladas em um arquivo para posteriormente serem rotuladas manualmente em:

- "CORRECT": se a resposta tiver sido corretamente respondida
- "INCORRECT": se a resposta for textualmente coerente, mas incorreta em seu conteúdo
- "NONSENSE": se a resposta for textualmente incorente (ex.: números aleatórios, repetição de palavras etc, tokens aleatórios etc)

In [ ]:
# Criando o cliente de Q&A
qa_client_google = QuestionAnsweringClient(llm_model=llm_model_llama, vectorstore=vectorstore_google, search_type="similarity", search_kwargs={"k":10})
qa_client_st = QuestionAnsweringClient(llm_model=llm_model_llama, vectorstore=vectorstore_st, search_type="similarity", search_kwargs={"k":10})
qa_client_st_mini = QuestionAnsweringClient(llm_model=llm_model_llama, vectorstore=vectorstore_st_mini, search_type="similarity", search_kwargs={"k":10})
qa_client_base = QuestionAnsweringClient(llm_model=llm_model_llama, vectorstore=None)

In [ ]:
# Escolhendo 100 perguntas aleatoriamente
df_question_answers = squad_dataset[squad_dataset.is_impossible == False].sample(n=100, random_state=42)
print(f'Shape do DataFrame de perguntas: {df_question_answers.shape}')

In [ ]:
# Número de perguntas por área de conhecimento:
plt.figure(figsize=(10,6))
p = sns.barplot(x=df_question_answers['title'].value_counts().index, y=df_question_answers['title'].value_counts(), color='green')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=90)
plt.xlabel('Área de conhecimento')
plt.ylabel('Número de perguntas')
plt.title('Número de perguntas selecionadas por área de conhecimento')
for c in p.containers:
  labels = [int(v.get_height()) for v in c]
  p.bar_label(c, labels=labels, label_type='edge')
plt.show()

In [ ]:
# Gerando respostas com RAG
print('Começando...')
print('Respondendo perguntas com RAG baseado no modelo de embeddings Gecko da Google')
df_question_answers['answer_google'] = df_question_answers['question'].apply(lambda x: qa_client_google.answer_question_with_rag(question=x))

print('Respondendo perguntas com RAG baseado no modelo de embeddings all-mpnet-base-v2 da Sentence Transformers ')
df_question_answers['answer_st'] = df_question_answers['question'].apply(lambda x: qa_client_st.answer_question_with_rag(question=x))

print('Respondendo perguntas com RAG baseado no modelo de embeddings all-MiniLM-L6-v2 da Sentence Transformers ')
df_question_answers['answer_st_mini'] = df_question_answers['question'].apply(lambda x: qa_client_st_mini.answer_question_with_rag(question=x))
print('Fim!')

In [ ]:
# Gerando as respostas sem usar RAG
df_question_answers['answer_without_rag'] = df_question_answers['question'].apply(lambda x: qa_client_base.answer_question(question=x))

In [ ]:
df_question_answers.tail().T

In [ ]:
# Obtendo os documentos recuperados para cada pergunta
df_question_answers['retrieved_docs_google'] = df_question_answers['question'].apply(lambda x: [doc.metadata for doc in qa_client_google.retriever.invoke(x)])
df_question_answers['retrieved_docs_st'] = df_question_answers['question'].apply(lambda x: [doc.metadata for doc in qa_client_st.retriever.invoke(x)])
df_question_answers['retrieved_docs_st_mini'] = df_question_answers['question'].apply(lambda x: [doc.metadata for doc in qa_client_st_mini.retriever.invoke(x)])

In [ ]:
# Extraindo os IDs dos documentos recuperados
df_question_answers['retrieved_docs_ids_google'] = df_question_answers['retrieved_docs_google'].map(lambda x: [doc['doc_num'] for doc in x])
df_question_answers['retrieved_docs_ids_st'] = df_question_answers['retrieved_docs_st'].map(lambda x: [doc['doc_num'] for doc in x])
df_question_answers['retrieved_docs_ids_st_mini'] = df_question_answers['retrieved_docs_st_mini'].map(lambda x: [doc['doc_num'] for doc in x])

In [ ]:
# Extraindo os títulos dos documentos recuperados
df_question_answers['retrieved_docs_titles_google'] = df_question_answers['retrieved_docs_google'].map(lambda x: ' '.join([doc['title'] for doc in x]))
df_question_answers['retrieved_docs_titles_st'] = df_question_answers['retrieved_docs_st'].map(lambda x: [doc['title'] for doc in x])
df_question_answers['retrieved_docs_titles_st_mini'] = df_question_answers['retrieved_docs_st_mini'].map(lambda x: [doc['title'] for doc in x])

In [ ]:
# Criando uma coluna com flag para determinar se conseguiu recuperar o documento certo
df_question_answers['found_correct_doc_google'] = df_question_answers.apply(lambda x: x['doc_num'] in x['retrieved_docs_ids_google'], axis=1)
df_question_answers['found_correct_doc_st'] = df_question_answers.apply(lambda x: x['doc_num'] in x['retrieved_docs_ids_st'], axis=1)
df_question_answers['found_correct_doc_st_mini'] = df_question_answers.apply(lambda x: x['doc_num'] in x['retrieved_docs_ids_st_mini'], axis=1)

In [ ]:
# Definindo um texto de respostas aceitáveis para melhor visualização
df_question_answers['expected_answers_tx'] = df_question_answers['answers'].map(lambda x: ' <OR> '.join([answer['text'] for answer in x]))

In [ ]:
# Exporta o arquivo para Excel para fazer a anotação manual das respostas
df_question_answers.to_excel('questions_answers.xlsx')

## Avaliando resultados



In [ ]:
# Importa o arquivo com as anotações das respostas
df_qa_annotated = pd.read_excel('questions_answers_annotated.xlsx')

In [ ]:
columns = ['question',
           'title',
           'found_correct_doc_google',
           'found_correct_doc_st',
           'found_correct_doc_st_mini',
           'result_answer_without_rag',
           'result_answer_google',
           'result_answer_st',
           'result_answer_st_mini',
           'retrieved_docs_ids_google',
           'retrieved_docs_ids_st',
           'retrieved_docs_ids_st_mini',
           'retrieved_docs_titles_google',
           'retrieved_docs_titles_st',
           'retrieved_docs_titles_st_mini'
           ]
df_qa_annotated = df_qa_annotated[columns]

In [ ]:
df_qa_annotated.head()

In [ ]:
# Criando um dataframe mais adequado para a plotagem
setup_names = ['without_rag', 'google', 'st', 'st_mini']
results_array = np.array([0, 0, 0, 0])
for name in setup_names:
  array = df_qa_annotated['result_answer_' + name].value_counts().to_numpy()
  if name == 'without_rag':
    array = np.append(array, [0])
  else:
    docs_found = df_qa_annotated['found_correct_doc_' + name].value_counts().to_numpy()[0]
    array = np.append(array, [docs_found])
  results_array = np.vstack([results_array, array])

df_plot = pd.DataFrame(results_array, columns=['correct_count', 'incorrect_count', 'nonsense_count', 'doc_found_count']).drop(0)
df_plot['name'] = setup_names
df_plot.head()

In [ ]:
# Plotando o número de respostas corretas para cada configuração
plt.figure(figsize=(10,6))
p = sns.barplot(df_plot, x=['Sem RAG', 'Gecko', 'all-mpnet-base-v2', 'all-MiniLM-L6-v2'], y='correct_count')

for c in p.containers:
  labels = [int(v.get_height()) for v in c]
  p.bar_label(c, labels=labels, label_type='edge')

p.set_title('Número de respostas corretas por configuração')
plt.ylabel('Número de respostas')
plt.xlabel('Configuração')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Plotando o número de respostas sem sentido para cada configuração
plt.figure(figsize=(10,6))
p = sns.barplot(df_plot, x=['Sem RAG', 'Gecko', 'all-mpnet-base-v2', 'all-MiniLM-L6-v2'], y='nonsense_count')

for c in p.containers:
  labels = [int(v.get_height()) for v in c]
  p.bar_label(c, labels=labels, label_type='edge')

p.set_title('Número de respostas sem sentido por configuração')
plt.ylabel('Número de respostas sem sentido')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Plotando o número de respostas incorretas para cada configuração
plt.figure(figsize=(10,6))
p = sns.barplot(df_plot, x=['Sem RAG', 'Gecko', 'all-mpnet-base-v2', 'all-MiniLM-L6-v2'], y='incorrect_count')

for c in p.containers:
  labels = [int(v.get_height()) for v in c]
  p.bar_label(c, labels=labels, label_type='edge')

p.set_title('Número de respostas incorretas por configuração')
plt.ylabel('Número de respostas sem sentido')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Plotando o número de perguntas cujo documento foi corretamente recuperado pelo RAG
plt.figure(figsize=(10,6))
p = sns.barplot(df_plot[1:4], x=['Gecko','all-mpnet-base-v2', 'all-MiniLM-L6-v2'], y='doc_found_count')

for c in p.containers:
  labels = [int(v.get_height()) for v in c]
  p.bar_label(c, labels=labels, label_type='edge')

p.set_title('Número de perguntas cujo documento foi corretamente encontrado')
plt.ylabel('Número de perguntas')
plt.xlabel('Configuração')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()